# Ames Housing DataSet

* The Ames Housing dataset was compiled by Dean De Cock for use in data science education. With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this dataset can be used to predict the final price of each home.

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Load and Inspect Data

In [ ]:
def load_housing_data(file_path):
    HOUSING_PATH = 'data/'
    csv_path = os.path.join(HOUSING_PATH, file_path)
    return pd.read_csv(csv_path)

In [ ]:
train = load_housing_data('train.csv')
test = load_housing_data('test.csv')
print(train.shape, test.shape)
train.head()

In [ ]:
# Data type of each feature
# nominal : categorical features with no clear ordering
# ordinal : has an ordering, ie. 0-10, A-Z scale

cat_feats_nominal = ['MSSubClass', 'MSZoning', 'Neighborhood', 'Condition1', 'Condition2', 'HouseStyle', 'CentralAir', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'Electrical', 'MasVnrType', 'Exterior1st', 'Exterior2nd', 'Heating', 'Foundation']
cat_feats_ordinal = ['Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'BldgType', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','PavedDrive', 'Fence']

numeric_feats_cont= ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'TotalSF', 'TotalSF1', 'YrBltAndRemod', 'TotalBathrooms', 'TotalPorchSF']
numeric_feats_ordinal= ['OverallQual', 'OverallCond']
numeric_feats_discrete= ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars','haspool', 'has2ndfloor', 'hasgarage', 'hasbsmt', 'hasfireplace']

In [ ]:
train.describe()

### Drop unnecessary ID attribute

In [ ]:
train.drop('Id', inplace= True, axis=1)
test.drop('Id', inplace= True, axis=1)
print(train.shape, test.shape)

## Data Visualization

### Histogram

In [ ]:
import matplotlib.pyplot as plt
figure = train.hist(bins=50, figsize=(30,20))
plt.show()

### Correlation Matrix

In [ ]:
import seaborn as sns
train_corr = train.corr(method= 'pearson') # Compute pairwise correlation of columns, excluding NA/null values. pearson : standard correlation coefficient
f, ax = plt.subplots(figsize=(25, 25))

# Generate a mask to hide  the upper triangle
mask = np.triu(np.ones_like(train_corr, dtype=bool))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

ax = sns.heatmap(train_corr, vmin=-1, vmax=1, mask=mask, cmap=cmap, center=0, annot = True, square=True, linewidths=.5, cbar_kws= {"shrink": .5, 'orientation': 'vertical'}) 

The features that highly correlate with `SalePrice` are: 
- `OverallQual` (0.79)
- `GrLivArea` (0.71), `GarageArea` (0.62), `TotalBsmtSF` (0.61), `1stFlrSF` (0.61)
- `YearBuilt` (0.52), `YearRemodAdd` (0.51)
- `FullBath`(0.56), `TotRmsAbvGrd` (0.53), `GarageCars` (0.64)

Let's take a loot at these attributes separately with respect to `SalePrice`...

### `OveralQual` vs. `SalePrice`

* Correlation coefficient is 0.79
* Since `OverallQual` rates the overall material and finish and ranges from 10 (Very Excellent) to 1 (Very Poor), we can use boxplot and violinplot for visualization

In [ ]:
figure, ax = plt.subplots(1,2, figsize = (24,10))
sns.boxplot(data=train, x= 'OverallQual', y='SalePrice', ax = ax[0])
sns.violinplot(data=train, x= 'OverallQual', y='SalePrice', ax = ax[1])
ax[0].set_title('OverallQual vs. SalePrice (Correlation coefficient: 0.79)')
ax[1].set_title('OverallQual vs. SalePrice (Correlation coefficient: 0.79)')
plt.show()

### `TotalBsmtSF`, `1stFlrSF`, `GrLivArea`, `GarageArea` vs. `SalePrice`

* Correlation coefficients are `TotalBsmtSF` (0.61), `1stFlrSF` (0.61), `GrLivArea` (0.71), `GarageArea` (0.62) 
* Since all of these features contains surface areas in square feet, we can use regplots to plot the data

In [ ]:
figure, ax = plt.subplots(2,2, figsize = (24,8))
figure.tight_layout(pad=5.0)

sns.regplot(data=train, x = 'TotalBsmtSF', y='SalePrice', scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'}, ax = ax[0,0])
ax[0,0].set_title('TotalBsmtSF vs SalePrice (Correlation coefficient: 0.61)', fontsize = 12)

sns.regplot(data=train, x = '1stFlrSF', y='SalePrice', scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'}, ax = ax[0,1])
ax[0,1].set_title('1stFlrSF vs SalePrice (Correlation coefficient: 0.61)', fontsize = 12)

sns.regplot(data=train, x = 'GrLivArea', y='SalePrice', scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'}, ax = ax[1,0])
ax[1,0].set_title('\nGrLivArea vs SalePrice (Correlation coefficient: 0.71)', fontsize = 12)

sns.regplot(data=train, x = 'GarageArea', y='SalePrice', scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'}, ax = ax[1,1])
ax[1,1].set_title('\nGarageArea vs SalePrice (Correlation coefficient: 0.62)', fontsize = 12)

plt.show()

### `YearBuilt`, `YearRemodAd` vs. `SalePrice`
* Correlation coefficients are 0.52 and 0.51 respectively
* Since these contains year values, we can use the barplot and regplot to visualize the data

In [ ]:
figure, ax = plt.subplots(2,1, figsize = (24,10))
figure.tight_layout(pad=5.0) # To increase the space between subplots

sns.barplot(ax = ax[0], x='YearBuilt', y="SalePrice", data = train)
ax[0].set(xlabel="YearBuilt", ylabel = "SalePrice")
ax[0].set_title('YearBuilt vs SalePrice (Correlation coefficient: 0.52)')
ax[0].set_xticklabels(ax[0].get_xticklabels(),rotation=90)

sns.barplot(ax = ax[1], x='YearRemodAdd', y="SalePrice", data = train)
ax[1].set(xlabel="YearRemodAdd", ylabel = "SalePrice")
ax[1].set_title('YearRemodAdd vs SalePrice (Correlation coefficient: 0.51)')
ax[1].set_xticklabels(ax[1].get_xticklabels(),rotation=90)

plt.show()

In [ ]:
figure, ax = plt.subplots(1,2, figsize = (24,6))
sns.regplot(data=train, x = 'YearBuilt', y='SalePrice', scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'}, ax = ax[0]) # scatter_kws and line_kws used to pass additional keyword argument to change transparancy and line color
ax[0].set_title('YearBuilt vs SalePrice (Correlation coefficient: 0.52)', fontsize = 12)

sns.regplot(data=train, x = 'YearRemodAdd', y='SalePrice', scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'}, ax = ax[1])
ax[1].set_title('YearRemodAdd vs SalePrice (Correlation coefficient: 0.51)', fontsize = 12)

plt.show()

### `FullBath`, `TotRmsAbvGrd`, `GarageCars` vs. `SalePrice`

-  Correlation coefficients are `FullBath`(0.56), `TotRmsAbvGrd` (0.53), `GarageCars` (0.64)
- These features contain discrete values so we will use boxplot and violinplots for these

In [ ]:
figure, ax = plt.subplots(3,2, figsize = (24,12))
figure.tight_layout(pad=4.0) # To increase the space between subplots

sns.boxplot(data=train, x = 'FullBath', y='SalePrice', ax = ax[0,0])
sns.violinplot(data=train, x = 'FullBath', y='SalePrice', ax = ax[0,1])
ax[0,0].set_title('FullBath vs SalePrice (Correlation coefficient: 0.56)', fontsize = 12)
ax[0,1].set_title('FullBath vs SalePrice (Correlation coefficient: 0.56)', fontsize = 12)

sns.boxplot(data=train, x = 'TotRmsAbvGrd', y='SalePrice', ax = ax[1,0])
sns.violinplot(data=train, x = 'TotRmsAbvGrd', y='SalePrice', ax = ax[1,1])
ax[1,0].set_title('TotRmsAbvGrd vs SalePrice (Correlation coefficient: 0.53)', fontsize = 12)
ax[1,1].set_title('TotRmsAbvGrd vs SalePrice (Correlation coefficient: 0.53)', fontsize = 12)

sns.boxplot(data=train, x = 'GarageCars', y='SalePrice', ax = ax[2,0])
sns.violinplot(data=train, x = 'GarageCars', y='SalePrice', ax = ax[2,1])
ax[2,0].set_title('GarageCars vs SalePrice (Correlation coefficient: 0.64)', fontsize = 12)
ax[2,1].set_title('GarageCars vs SalePrice (Correlation coefficient: 0.64)', fontsize = 12)

plt.show()

## Remove Outliers

* We will remove outliers outside of the minimum and maximum threshold 

In [ ]:
min_percentile= 0.001
max_percentile= 0.999
# Use numeric features
features = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea','BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
            'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
            'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
target= 'SalePrice'
nrows= int(np.ceil(len(features)/2))
ncols= 2 

def remove_outliers(inline_delete=True):
    global train
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=((24,nrows * 6)))
    outliers = []
    cnt = 0
    for row in range(0, nrows):
        for col in range(0, ncols):
            min_threshold, max_threshold = train[features[cnt]].quantile([min_percentile, max_percentile])
            df_outliers = train[(train[features[cnt]] < min_threshold) | (train[features[cnt]] > max_threshold)]
            outliers += df_outliers.index.tolist()
            
            ax[row][col].scatter(x=train[features[cnt]], y=train[target])
            
            ax[row][col].scatter(x= df_outliers[features[cnt]],  y=df_outliers[target], marker ="o", edgecolor ="red", s = 100)
            ax[row][col].set_xlabel(features[cnt])
            ax[row][col].set_ylabel(target)
            ax[row][col].set_title('Outlier detection for feature ' + features[cnt])
            
            if inline_delete:
                train = train.drop(df_outliers.index.tolist())
                train.reset_index(drop=True, inplace=True)
                
            cnt += 1
            if cnt >= len(features):
                break
                    
            
    plt.show()
                
    unique_outliers= list(set(outliers))
                              
    if not inline_delete:
        train = train.drop(unique_outliers)
        train.reset_index(drop = True, inplace = True)
        
remove_outliers(inline_delete=False)

## Process Data

### Separate target feature (`SalePrice`) from training set

In [ ]:
y_train = train.SalePrice
train.drop(['SalePrice'], axis=1, inplace=True)

print(train.shape, test.shape)
train.head()

### Delete Unnecessary Features

* Drop `Utilities`, `Street`, and `PoolQC` features
* `Utilities` contains only one type of utility and is not useful for the model
* `Street` contains unbalanced data of type road access
* `PoolQC` has mostly missing data

In [ ]:
train.drop(['Utilities', 'Street', 'PoolQC'], axis=1, inplace=True)
test.drop(['Utilities', 'Street', 'PoolQC'], axis=1, inplace=True)

In [ ]:
print(train.shape, test.shape)
train.head()

### Fix DataTypes
* `MSSubClass`: Identifies the type of dwelling involved in the sale. Its data type is int64 and values are incremental order starting from 20 upto 190. If we keep it as it is then our model may give more importance to MSSubClass 190 houses over MSSubClass 20 houses. In order to avoid that we will change its data type to 'str' and treat this as categorical variable.
* `YrSold`: Contains year values like 2008, 2007, 2006, 2009, 2010. Since we have sufficient data for each value we will change its data type to 'str' and treat this as categorical variable.
* `MoSold`: Since we have sufficient data for each value we will change its data type to 'str' and treat this as categorical variable.
* We are not changing the data type of `YearBuilt`, `YearRemodAdd` and `GarageYrBlt` to 'str'. They have linear relationship with `SalePrice`, model will benefit from this relationship instaed of converting it into categories. (Also we dont have sufficient training examples for each unique value)

In [ ]:
for col in ('MSSubClass', 'YrSold', 'MoSold'):
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

### Check Missing Values

In [ ]:
train_col_na = train.columns[train.isnull().any()]
test_col_na = test.columns[test.isnull().any()]

# Get missing value count in each column
train_na_cnt = train[train_col_na].isnull().sum()
test_na_cnt = test[test_col_na].isnull().sum()

# Get missing values percentage for each column
train_na = (train[train_col_na].isnull().sum()/len(train)) * 100
test_na = (test[test_col_na].isnull().sum()/len(test)) * 100


display_na = pd.DataFrame({'Train Null Val': train_na_cnt, 'Train Null Val %': train_na, 'Test Null Val': test_na_cnt, 'Test Null Val %': test_na})
display_na = display_na.sort_values(by='Train Null Val %', ascending=False)
display_na

In [ ]:
sns.barplot(x=display_na.index, y='Train Null Val %', data=display_na)
plt.xticks(rotation = 90) 
plt.title('Train Column name with null value %')
plt.show()

In [ ]:
sns.barplot(x=display_na.index, y='Test Null Val %', data=display_na)
plt.xticks(rotation = 90) 
plt.title('Test Column name with null value %')
plt.show()

## Impute Missing Values

### Replace Missing Categorical Features with 'None'

In [ ]:
for col in ('MiscFeature', 'Alley','FireplaceQu', 'GarageFinish', 'GarageQual', 'Fence', 'GarageType', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'MSSubClass'):
    train[col] = train[col].fillna('None')
    test[col] = test[col].fillna('None')

    print(f'Feature: {col}, Null Count: {train[col].isnull().sum()}, Unique Values: {train[col].unique()}')

### Replace with Median

* `LotFrontage`: Linear feet of street connected to property. Since the area of each street connected to the house property most likely have a similar area to other houses in its neighborhood, we can fill in missing values by the median LotFrontage of the neighborhood.

* Since `LotFrontage` contains continuous data we are taking 'median' value

In [ ]:
train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

In [ ]:
test['LotFrontage'] = test.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

### Replace Missing Numerical Values with '0'

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'):
    train[col] = train[col].fillna(0)
    test[col] = test[col].fillna(0)

### Impute with Specific Value

* As per the data description, assume 'Typ' home functionality unless deductions are warranted.

In [ ]:
train['Functional'] = train['Functional'].fillna('Typ')
test['Functional'] = test['Functional'].fillna('Typ')

### Replace with Mode

* For features with low percentage of null values, we will use the most frequent value to replace the missing value

In [ ]:
for col in ('MSZoning','Electrical','KitchenQual','Exterior1st','Exterior2nd', 'SaleType'):
    train[col] = train[col].fillna(train[col].mode()[0])
    test[col] = test[col].fillna(test[col].mode()[0])

One last check...

In [ ]:
print(f'Count of train null values: {train.isnull().sum().sum()}')
print(f'Count of test null values: {test.isnull().sum().sum()}')

fig, ax = plt.subplots(1, 2, figsize = (10, 3))
sns.heatmap(train.isnull(), ax=ax[0])
sns.heatmap(test.isnull(), ax=ax[1])

## Remove Multicollinear Features

* Method `get_highest_vif_feature()` will find all the features with Variance Inflation Factor (VIF) more than threshold value and return the feature with highest VIF
* Instead of removing all the features with VIF more than threshold value we will drop the feature with highest VIF value and recalculate the VIF for all the remaining features.
* We will repeat this step until no remaining features have a VIF larger than threshold value.
* Once we get the final list of multicolenear features we will verify it with Correlation Matrix results and only drop those features, which we are 100% sure won't help in modeling.

In [ ]:
!pip install statsmodels

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def get_highest_vif_feature(df, thresh=5):
    '''
    Ref: https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python
    
    Calculates VIF each feature in a pandas dataframe
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param thresh: the max VIF value before the feature is removed from the dataframe
    :return: dataframe with features removed
    '''
   
    const = add_constant(df)
    print(f'Shape of data after adding const column: {const.shape}')
    cols = const.columns
    
    # Calculating VIF for each feature
    vif_df = pd.Series([ (variance_inflation_factor(const.values, i)) for i in range(const.shape[1]) ], index= const.columns).to_frame()
    
    vif_df = vif_df.sort_values(by=0, ascending=False).rename(columns={0: 'VIF'})
    vif_df = vif_df.drop('const')
    vif_df = vif_df[vif_df['VIF'] > thresh]

    if vif_df.empty:
        print('DataFrame is empty!')
        return None
    else:
        print(f'\nFeatures above VIF threshold: {vif_df.to_dict()}')       
        # Feature with highest VIF value
        return list(vif_df.index)[0]
        print(f'Lets delete the feature with highest VIF value: {list(vif_df.index)[0]}')

In [ ]:
# Selecting only numeric features
print(f'Shape of input data: {train.shape}')
numeric_feats = train.dtypes[train.dtypes != "object"].index
print(f"Calculating VIF for {len(numeric_feats)} numerical features")

df_numeric = train[numeric_feats]
print(f'Shape of df_numeric: {df_numeric.shape}')
    
feature_to_drop = None
feature_to_drop_list = []
while True:
    feature_to_drop = get_highest_vif_feature(df_numeric, thresh=5)
    print(f'feature_to_drop: {feature_to_drop}')
    if feature_to_drop is None:
        print('No more features to drop!')
        break
    else:
        feature_to_drop_list.append(feature_to_drop)
        df_numeric = df_numeric.drop(feature_to_drop, axis=1)
        print(f'Feature {feature_to_drop} droped from df_numeric')

print(f'\nfeature_to_drop_list: {feature_to_drop_list}')

In [ ]:
# Selecting only numeric features
print(f'Shape of input data: {test.shape}')
numeric_feats = test.dtypes[test.dtypes != "object"].index
print(f"Calculating VIF for {len(numeric_feats)} numerical features")

df_numeric = test[numeric_feats]
print(f'Shape of df_numeric: {df_numeric.shape}')
    
feature_to_drop = None
feature_to_drop_list = []
while True:
    feature_to_drop = get_highest_vif_feature(df_numeric, thresh=5)
    print(f'feature_to_drop: {feature_to_drop}')
    if feature_to_drop is None:
        print('No more features to drop!')
        break
    else:
        feature_to_drop_list.append(feature_to_drop)
        df_numeric = df_numeric.drop(feature_to_drop, axis=1)
        print(f'Feature {feature_to_drop} droped from df_numeric')

print(f'\nfeature_to_drop_list: {feature_to_drop_list}')

Now lets compare the result of VIF with correlation matrix values.

* Features correlated with `SalePrice` are OverallQual(0.79), YearBuilt(0.52), YearRemodAdd(0.51), TotalBsmtSF(0.61), 1stFlrSF(0.61), GrLivArea(0.71), FullBath(0.56), TotRmsAbvGrd(0.53), GarageCars(0.64), GarageArea(0.62)
* Features not correlated with SalePrice are MSSubClass(-0.084), OverallCond(-0.078), BsmtFinSF1(-0.011), LowQualFinSF(-0.026), BsmtHalfBath(-0.017), KitchenAbvGrd(-0.14), EnclosedPorch(-0.13), MiscVal(-0.021), YrSold(-0.029)
* Even though area features are correlated, we dont want to delete them. These features are very usefull to predict the sales price. Infact next during feature engineering we are going to add few more area features by combining existing features. Now from the list of features predicted by VIF we can only delete `LowQualFinSF` for now.

In [ ]:
train = train.drop(['LowQualFinSF'], axis=1)
test = test.drop(['LowQualFinSF'], axis=1)

train.shape

## Feature Scaling

Let's check the skewness of the data...

In [ ]:
from scipy import stats

cat_feats = train.dtypes[train.dtypes == "object"].index
numeric_feats = train.dtypes[train.dtypes != "object"].index
print(f"Number of categorical features: {len(cat_feats)}, Numerical features: {len(numeric_feats)}")

train_skew_features = train[numeric_feats].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
train_skewness = pd.DataFrame({'Skew': train_skew_features})

print(f'Skew in numerical features. Shape of skewness: {train_skewness.shape}')
train_skewness.head(10)

In [ ]:
cat_feats = test.dtypes[test.dtypes == "object"].index
numeric_feats = test.dtypes[test.dtypes != "object"].index
print(f"Number of categorical features: {len(cat_feats)}, Numerical features: {len(numeric_feats)}")

test_skew_features = test[numeric_feats].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
test_skewness = pd.DataFrame({'Skew': test_skew_features})

print(f'Skew in numerical features. Shape of skewness: {test_skewness.shape}')
test_skewness.head(10)

* As we can see from above details that our data is skewed. Remember that Multivariate normality is one of the assumption for linear regression. Multivariate normality means that regression requires all its variables to be normal. By having skewed data we violate the assumption of normality.
* So we must choose the right method which will scale our features as well as maintain the multivariate normality.
* We will use a Box Cox transformation. A Box Cox transformation is a way to transform non-normal dependent variables into a normal shape. 

In [ ]:
from scipy.special import boxcox1p

train_high_skew = train_skew_features[train_skew_features > 0.5]
test_high_skew = test_skew_features[test_skew_features > 0.5]

train_skew_index = train_high_skew.index
test_skew_index = test_high_skew.index

for i in train_skew_index:
    train[i] = boxcox1p(train[i], stats.boxcox_normmax(train[i] + 1))
for j in test_skew_index:
    test[i] = boxcox1p(test[i], stats.boxcox_normmax(test[i] + 1))

## Features Engineering

* Since area related features are very important to determine the house price, we will create new feature by name 'TotalSF' by adding 'TotalBsmtSF', '1stFlrSF' and '2ndFlrSF'.
* Similarly we will create one more new feature by name 'TotalSF1' by adding 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF' and '2ndFlrSF'. Here 'BsmtFinSF1' and 'BsmtFinSF2' represent finished square feet of all area, thats why we are creating separate feature using it.
* Create new feature 'YrBltAndRemod' by adding 'YearBuilt' and 'YearRemodAdd'
* Create new feature 'TotalBathrooms' by adding all the bathrooms in the house.
* Create new feature 'TotalPorchSF' by adding all porch area.

In [ ]:
train['TotalSF'] = train['TotalBsmtSF'] + train['1stFlrSF'] + train['2ndFlrSF']
train['TotalSF1'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']

train['YrBltAndRemod']= train['YearBuilt'] + train['YearRemodAdd']

train['TotalBathrooms'] = (train['FullBath'] + (0.5 * train['HalfBath']) +
                               train['BsmtFullBath'] + (0.5 * train['BsmtHalfBath']))

train['TotalPorchSF'] = (train['OpenPorchSF'] + train['3SsnPorch'] +
                              train['EnclosedPorch'] + train['ScreenPorch'] +
                              train['WoodDeckSF'])

In [ ]:
test['TotalSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']
test['TotalSF1'] = test['BsmtFinSF1'] + test['BsmtFinSF2'] + test['1stFlrSF'] + test['2ndFlrSF']

test['YrBltAndRemod']= test['YearBuilt'] + test['YearRemodAdd']

test['TotalBathrooms'] = (test['FullBath'] + (0.5 * test['HalfBath']) +
                               test['BsmtFullBath'] + (0.5 * test['BsmtHalfBath']))

test['TotalPorchSF'] = (test['OpenPorchSF'] + test['3SsnPorch'] +
                              test['EnclosedPorch'] + test['ScreenPorch'] +
                              test['WoodDeckSF'])

Let's add new features based on the availability of the swimming pool, second floor, garage, basement and fireplace.

In [ ]:
train['haspool'] = train['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
train['has2ndfloor'] = train['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
train['hasgarage'] = train['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
train['hasbsmt'] = train['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
train['hasfireplace'] = train['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
test['haspool'] = test['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
test['has2ndfloor'] = test['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
test['hasgarage'] = test['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
test['hasbsmt'] = test['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
test['hasfireplace'] = test['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

## Encode Categorical Features

In [ ]:
# Manually label cat ordinal features
s = [train, test]

for dataset in s:
    dataset['Alley'].replace(to_replace = ['None', 'Grvl', 'Pave'], value = [0, 1, 2], inplace = True)
    dataset['LotShape'].replace(to_replace = ['Reg', 'IR1', 'IR2', 'IR3'], value = [3, 2, 1,0], inplace = True)
    dataset['LandContour'].replace(to_replace = ['Lvl', 'Bnk', 'Low', 'HLS'], value = [3, 2, 1,0], inplace = True)
    dataset['LotConfig'].replace(to_replace = ['Inside', 'FR2', 'Corner', 'CulDSac', 'FR3'], value = [0, 3, 1, 2, 4], inplace = True)
    dataset['LandSlope'].replace(to_replace = ['Gtl', 'Mod', 'Sev'], value = [2, 1, 0], inplace = True)
    dataset['BldgType'].replace(to_replace = ['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], value = [4, 3, 2, 1, 0], inplace = True)
    dataset['RoofStyle'].replace(to_replace = ['Gable', 'Hip', 'Gambrel', 'Mansard', 'Flat', 'Shed'], value = [4, 2, 3, 1, 5, 0], inplace = True)
    dataset['RoofMatl'].replace(to_replace = ['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'], value = [7, 6, 5, 4, 3, 2, 1, 0], inplace = True)
    dataset['ExterQual'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa'], value = [3, 2, 1, 0], inplace = True)
    dataset['ExterCond'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa', 'Po'], value = [4, 3, 2, 1, 0], inplace = True)
    dataset['BsmtQual'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa', 'None'], value = [4, 3, 2, 1, 0], inplace = True)
    dataset['BsmtCond'].replace(to_replace = ['Gd', 'TA', 'Fa', 'Po', 'None'], value = [4, 3, 2, 1, 0], inplace = True)
    dataset['BsmtExposure'].replace(to_replace = ['Gd', 'Av', 'Mn', 'No', 'None'], value = [4, 3, 2, 1, 0], inplace = True)
    dataset['BsmtFinType1'].replace(to_replace = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'None'], value = [6, 5, 4, 3, 2, 1, 0], inplace = True)
    dataset['BsmtFinType2'].replace(to_replace = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'None'], value = [6, 5, 4, 3, 2, 1, 0], inplace = True)
    dataset['HeatingQC'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa', 'Po'], value = [4, 3, 2, 1, 0], inplace = True)
    dataset['KitchenQual'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa'], value = [3, 2, 1, 0], inplace = True)
    dataset['Functional'].replace(to_replace = ['Typ', 'Min1', 'Min2', 'Mod',  'Maj1', 'Maj2', 'Sev'], value = [6, 5, 4, 3, 2, 1, 0], inplace = True)
    dataset['FireplaceQu'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'None'], value = [5, 4, 3, 2, 1, 0], inplace = True)
    dataset['GarageType'].replace(to_replace = ['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd', 'None'], value = [6, 5, 4, 3, 2, 1, 0], inplace = True)
    dataset['GarageFinish'].replace(to_replace = ['Fin', 'RFn', 'Unf', 'None'], value = [3, 2, 1, 0], inplace = True)
    dataset['GarageQual'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'None'], value = [5, 4, 3, 2, 1, 0], inplace = True)
    dataset['GarageCond'].replace(to_replace = ['Ex', 'Gd', 'TA', 'Fa',  'Po', 'None'], value = [5, 4, 3, 2, 1, 0], inplace = True)
    dataset['PavedDrive'].replace(to_replace = ['Y', 'P', 'N'], value = [2, 1, 0], inplace = True)
    dataset['Fence'].replace(to_replace = ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'None'], value = [4, 3, 2, 1, 0], inplace = True)

In [ ]:
# One-hot encoding for cat nominal features
train_cat_nominal_one_hot = pd.get_dummies(train[cat_feats_nominal], drop_first= True).reset_index(drop=True)
test_cat_nominal_one_hot = pd.get_dummies(test[cat_feats_nominal], drop_first= True).reset_index(drop=True)

final_train, final_test = train_cat_nominal_one_hot.align(test_cat_nominal_one_hot, join='inner', axis=1)  # inner join

In [ ]:
# Drop all categorical columns and concat with one hot encodings
train = train.drop(cat_feats_nominal, axis= 'columns')
test = test.drop(cat_feats_nominal, axis='columns')

train = pd.concat([train, final_train], axis='columns')
test = pd.concat([test, final_test], axis='columns')

In [ ]:
print(train.shape, test.shape)

The training dataset is looking good!!

## Target Variable Analysis and Transformation

In [ ]:
def distplot_probplot():
    fig, ax = plt.subplots(1,2, figsize= (15,5))
    
    #sns.distplot(y_train, label='test_label2', norm_hist='True', ax=ax[0])
    sns.distplot(y_train, fit=stats.norm,label='test_label2', ax = ax[0])
    stats.probplot(y_train, plot = ax[1])
    
    plt.show()
    
    (mu, sigma) = stats.norm.fit(y_train)
    print('mean= {:.2f}, sigma= {:.2f}, mode= {:.2f})'.format(mu, sigma, stats.mode(y_train)[0][0]))
    
def normality_stats():
    print(f"Skewness: {abs(y_train).skew()}")
    print(f"Kurtosis: {abs(y_train).kurt()}")
    
distplot_probplot()

In [ ]:
normality_stats()

`SalePrice` is right skewed. We can try log transformation to correct the data

In [ ]:
y_train = np.log1p(y_train)

distplot_probplot()

In [ ]:
normality_stats()

The data is finally ready for training!!

## Build Models

In [ ]:
!pip install mlxtend

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error

from sklearn import pipeline
from sklearn import preprocessing

from sklearn.linear_model import LinearRegression,ElasticNet, Lasso, BayesianRidge, Ridge
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import  StackingCVRegressor

In [ ]:
n_folds = 5
random_state = 42
kf = KFold(n_splits=n_folds, random_state=random_state, shuffle=True)

In [195]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, train, y_train, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

cv_scores = []
cv_std = []

#models = ['linear_reg','bayesian_ridge_reg','lasso_reg','elastic_net_reg','ridge_reg','svr_reg', 'gbr_reg', 'lgbm_reg',
  #                 'xgb_reg','stacking_cv_reg']


def score_model(model_reg):
    score_model_reg = cv_rmse(model_reg)
    print(f'score_model_reg => mean: {score_model_reg.mean()}, std: {score_model_reg.std()}')
    cv_scores.append(score_model_reg.mean())
    cv_std.append(score_model_reg.std())

### Linear Regression

In [254]:
linear_reg = LinearRegression()
score_model(linear_reg)

score_model_reg => mean: 379823607799.15564, std: 759647068323.0322


### Bayesian Ridge Regression

In [255]:
bayesian_ridge_reg = BayesianRidge(compute_score= False)
score_model(bayesian_ridge_reg)

score_model_reg => mean: 11959062486281.871, std: 23918124825288.492


### Ridge Regression 

In [256]:
ridge_reg = pipeline.Pipeline([("scaling", preprocessing.RobustScaler()),
                               ("ridge", Ridge(random_state= random_state))])

score_model(ridge_reg)

/Users/vynguyen/ml/my_env/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.37462e-46): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/vynguyen/ml/my_env/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.39328e-46): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/vynguyen/ml/my_env/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.45959e-46): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


score_model_reg => mean: 24381.20243838968, std: 784.4514847493151


/Users/vynguyen/ml/my_env/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.28804e-46): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


### Lasso Regression

In [ ]:
lasso_reg = pipeline.Pipeline([("scaling", preprocessing.RobustScaler()),
                               ("lasso", Lasso(alpha=0.0004,
                                               max_iter=1e7,
                                               tol= 0.001,
                                               random_state= random_state))])

score_model(lasso_reg)

### Elastic Net

In [85]:
elastic_net_reg = pipeline.Pipeline([("scaling", preprocessing.RobustScaler()),
                               ("elastic_net", ElasticNet(positive= True,
                                                          precompute=False,
                                                          selection='random',
                                                          max_iter=1e7,
                                                          tol= 0.001,
                                                          random_state= random_state))])

score_model(elastic_net_reg)

score_model_reg => mean: 0.36638681187316946, std: 0.011399735102939838


### SVM

In [86]:
svr_reg = pipeline.Pipeline([("scaling", preprocessing.RobustScaler()),
                               ("svr", svm.SVR(C= 46,
                                               epsilon= 0.009,
                                               gamma= 0.0003))])

score_model(svr_reg)

score_model_reg => mean: 0.11301981712560838, std: 0.010541678489037408


### Gradient Boosting Regression

In [87]:
gbr_reg = GradientBoostingRegressor(n_estimators=2500,
                                      learning_rate= 0.03,
                                      random_state = random_state)
    
score_model(gbr_reg)

score_model_reg => mean: 0.12036832672240023, std: 0.011396707570450363


### Stacking CV Regression

In [ ]:
estimators = ( linear_reg, svr_reg, bayesian_ridge_reg, ridge_reg, lasso_reg, elastic_net_reg, gbr_reg)
final_estimator = gbr_reg
    
stacking_cv_reg = StackingCVRegressor(regressors= estimators,
                                  meta_regressor = final_estimator,
                                  use_features_in_secondary= True,
                                  random_state= random_state)


score_model_reg = np.sqrt(-cross_val_score(stacking_cv_reg, train.values, y_train, scoring="neg_mean_squared_error", cv=kf))
print(f'score_model_reg => mean: {score_model_reg.mean()}, std: {score_model_reg.std()}')
cv_scores.append(score_model_reg.mean())
cv_std.append(score_model_reg.std())

## Submission

In [88]:
def submit(model):
    model.fit(train, y_train)
    preds = model.predict(test)
    submission = {'Id': test.Id, 'SalePrice': preds}
    submission = pd.DataFrame(data=submission)
    submission.to_csv(type(model).__name__ + '_submission', index=False)
    print(submission)

In [90]:
submit(gbr_reg)

ValueError: X has 200 features, but DecisionTreeRegressor is expecting 207 features as input.